In [1]:
import sys
if '../working' not in sys.path: 
    sys.path.append('../working')

!sage -preparse ../working/normalizer.sage
!mv ../working/normalizer.sage.py ../working/normalizer.py

!sage -preparse ../working/self_similar.sage
!mv ../working/self_similar.sage.py ../working/self_similar.py


In [2]:
from normalizer import *
from self_similar import *
prepare_gap_env()

In [30]:
normalizers(12, dim=2)

[
[ 0 x0]  [  0  x0]  [-x0  x0]  [-x0   0]  [ x0  x0]  [ x0 -x0]
[x0  0], [-x0   0], [ x0  x0], [  0  x0], [-x0  x0], [ x0  x0],

[-x0 -x0]  [x0  0]
[-x0  x0], [ 0 x0]
]

In [3]:
G = gap(f'SpaceGroupOnLeftIT(2, 12)')
gens = G.GeneratorsOfGroup()
gens = [matrix(QQ, el) for el in gens]
gens

[
[-1  0  0]  [ 0 -1  0]  [ -1   0 1/2]  [1 0 1]  [1 0 0]
[ 0 -1  0]  [ 1  0  0]  [  0   1 1/2]  [0 1 0]  [0 1 1]
[ 0  0  1], [ 0  0  1], [  0   0   1], [0 0 1], [0 0 1]
]

In [4]:
alpha = {}
for x in gens:
    for y in gens: 
        el = x * y
        g = el[:2, :2]
        t = el[:2, 2]
        if str(g) not in alpha:
            alpha[str(g)] = t
            print(el)
            print()

[1 0 0]
[0 1 0]
[0 0 1]

[ 0  1  0]
[-1  0  0]
[ 0  0  1]

[   1    0 -1/2]
[   0   -1 -1/2]
[   0    0    1]

[-1  0 -1]
[ 0 -1  0]
[ 0  0  1]

[   0   -1 -1/2]
[  -1    0  1/2]
[   0    0    1]

[ 0 -1  0]
[ 1  0  1]
[ 0  0  1]

[  0   1 1/2]
[  1   0 1/2]
[  0   0   1]

[  -1    0 -1/2]
[   0    1  1/2]
[   0    0    1]



In [5]:
P = G.PointGroup()
P = [matrix(QQ, el) for el in P.AsList()]
P

[
[1 0]  [-1  0]  [ 0  1]  [ 0 -1]  [ 1  0]  [-1  0]  [0 1]  [ 0 -1]
[0 1], [ 0 -1], [-1  0], [ 1  0], [ 0 -1], [ 0  1], [1 0], [-1  0]
]

In [25]:
var('a b')

(a, b)

In [59]:
A = matrix(QQ, [[-2, 2], [2, 2]])
x = matrix([[a], [b]])
E = matrix(QQ, [[1, 0], [0, 1]])

In [60]:
for p in P: 
    assert str(A.inverse() * p * A) in alpha, 'not in normalizer'

In [61]:
t = matrix(QQ, [[0], [0]])
t = block_matrix([[A, t], [0, 1]])
self_similar(12, t, dim=2, gen_alphabet=True)

ValueError: Bad matrix T, there is no virtual endomorphism

In [62]:
conds = []
variables = []
coeffs = []
for i, g in enumerate(P): 
    conj = A.inverse() * g * A
    condition = A.inverse() * alpha[str(g)] -  alpha[str(conj)]
    sym_res = (conj - E) * x
    print(sym_res)
    print(condition)
    print()

    conds.append(sym_res[0][0] - (condition[0][0] + var(f'n{i}')))
    cond_coeffs = {}
    for variable in conds[-1].arguments(): 
        cur_coeff = conds[-1].coefficients(variable, sparse=False)
        cond_coeffs[variable] = cur_coeff[1]
    coeffs.append(cond_coeffs)
    
    conds.append(sym_res[1][0] - (condition[1][0] + var(f'm{i}')))
    cond_coeffs = {}
    for variable in conds[-1].arguments(): 
        cur_coeff = conds[-1].coefficients(variable, sparse=False)
        cond_coeffs[variable] = cur_coeff[1]
    coeffs.append(cond_coeffs)
    
    variables.append(var(f'n{i}'))
    variables.append(var(f'm{i}'))

[0]
[0]
[0]
[0]

[-2*a]
[-2*b]
[ 5/4]
[-1/4]

[-a - b]
[ a - b]
[ 0]
[-1]

[-a + b]
[-a - b]
[1/4]
[1/4]

[-a - b]
[-a - b]
[ 1/2]
[-3/4]

[-a + b]
[ a - b]
[-1/4]
[-1/2]

[-2*a]
[   0]
[ 1/2]
[-1/4]

[   0]
[-2*b]
[3/4]
[1/2]



In [63]:
def get_constant(express): 
    while not isinstance(express, Integer) and not express.is_constant(): 
        express = express.coefficients(express.arguments()[0], sparse=False)[0]
    return express

for cof, con in zip(coeffs, conds): 
    cof['constant'] = get_constant(con)
coeffs

[{n0: -1, 'constant': 0},
 {m0: -1, 'constant': 0},
 {a: -2, n1: -1, 'constant': -5/4},
 {b: -2, m1: -1, 'constant': 1/4},
 {a: -1, b: -1, n2: -1, 'constant': 0},
 {a: 1, b: -1, m2: -1, 'constant': 1},
 {a: -1, b: 1, n3: -1, 'constant': -1/4},
 {a: -1, b: -1, m3: -1, 'constant': -1/4},
 {a: -1, b: -1, n4: -1, 'constant': -1/2},
 {a: -1, b: -1, m4: -1, 'constant': 3/4},
 {a: -1, b: 1, n5: -1, 'constant': 1/4},
 {a: 1, b: -1, m5: -1, 'constant': 1/2},
 {a: -2, n6: -1, 'constant': -1/2},
 {m6: -1, 'constant': 1/4},
 {n7: -1, 'constant': -3/4},
 {b: -2, m7: -1, 'constant': -1/2}]

In [66]:
coeff_matrix = []
const_matrix = []
for cof in coeffs: 
    coeff_row = []
    for variable in [a, b] + variables: 
        coeff_row.append(cof.get(variable, 0))
    coeff_matrix.append(coeff_row)
    const_matrix.append([cof.get('constant', 0)])
    
coeff_matrix = matrix(QQ, coeff_matrix)
int_coeff_mtx = matrix(ZZ, coeff_matrix * 4)

const_matrix = matrix(QQ, const_matrix)
int_const_mtx = matrix(ZZ, const_matrix * 4)

int_coeff_mtx

[ 0  0 -4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0 -4  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[-8  0  0  0 -4  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0 -8  0  0  0 -4  0  0  0  0  0  0  0  0  0  0  0  0]
[-4 -4  0  0  0  0 -4  0  0  0  0  0  0  0  0  0  0  0]
[ 4 -4  0  0  0  0  0 -4  0  0  0  0  0  0  0  0  0  0]
[-4  4  0  0  0  0  0  0 -4  0  0  0  0  0  0  0  0  0]
[-4 -4  0  0  0  0  0  0  0 -4  0  0  0  0  0  0  0  0]
[-4 -4  0  0  0  0  0  0  0  0 -4  0  0  0  0  0  0  0]
[-4 -4  0  0  0  0  0  0  0  0  0 -4  0  0  0  0  0  0]
[-4  4  0  0  0  0  0  0  0  0  0  0 -4  0  0  0  0  0]
[ 4 -4  0  0  0  0  0  0  0  0  0  0  0 -4  0  0  0  0]
[-8  0  0  0  0  0  0  0  0  0  0  0  0  0 -4  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -4  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -4  0]
[ 0 -8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -4]

In [67]:
int_const_mtx

[ 0]
[ 0]
[-5]
[ 1]
[ 0]
[ 4]
[-1]
[-1]
[-2]
[ 3]
[ 1]
[ 2]
[-2]
[ 1]
[-3]
[-2]

In [68]:
B, U, V = int_coeff_mtx.smith_form()

success = True
check = U * int_const_mtx
for i in range(len(B.rows())): 
    if check[i][0] % B[i][i]: 
        success = False 
        break 
success

False

In [69]:
B

[4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0]

In [70]:
check

[-2]
[-3]
[ 1]
[-2]
[ 2]
[ 1]
[ 3]
[-2]
[-1]
[-1]
[ 4]
[ 0]
[ 1]
[-5]
[ 0]
[ 0]